In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# df_meps = pd.read_csv('./data/silver/mep_membership_timeline.csv')

df_questions = pd.read_csv("./data/silver/questions_topics.csv", index_col=0)


questions_raw = pd.read_csv("./data/questions_details_2.csv")
questions_raw["id"] = questions_raw["id"].apply(lambda x: x.split("/")[-1])
questions_raw["creator"] = questions_raw["creator"].apply(
    lambda x: eval(x) if isinstance(x, str) else x
)
questions_raw = questions_raw.explode("creator")
questions_raw = questions_raw.drop_duplicates()

In [3]:
df = df_questions.reset_index().merge(questions_raw, on="id", how="left")

df = df[[True if 'person' in str(p) else False for p in df['creator']]]

In [4]:
df_descriptives = df.describe().T.sort_index()
df_descriptives

,count,mean,std,min,25%,50%,75%,max
agriculture,123934.0,0.194112,0.244546,0.000041,0.037912,0.103367,0.233060,0.999335
economics and trade,123934.0,0.320768,0.284428,0.000042,0.079717,0.230730,0.503391,0.998896
education,123934.0,0.151163,0.181124,0.000037,0.040083,0.095900,0.187657,0.999166
environment and climate,123934.0,0.318351,0.279954,0.000050,0.094470,0.230924,0.463007,0.998911
foreign and security affairs,123934.0,0.288059,0.271137,0.000075,0.069461,0.190847,0.438534,0.999188
health,123934.0,0.384880,0.298325,0.000092,0.135382,0.309944,0.589929,0.998705
human rights,123934.0,0.475546,0.318099,0.000138,0.186734,0.417587,0.780818,0.999586
identifierYear,123934.0,2018.449780,3.058206,2008.000000,2016.000000,2018.000000,2021.000000,2025.000000
infrastructure and industry,123934.0,0.298055,0.203283,0.000100,0.140985,0.263171,0.415619,0.997486
technology,123934.0,0.280681,0.221948,0.000164,0.124186,0.216449,0.360546,0.999130


# Categorizar temas

In [24]:
CUT = .5

topic_vars = df_descriptives[df_descriptives['max'] <= 1].index.tolist()

for var in topic_vars:
    df[f"infered_topic_{var}"] = df[var].apply(lambda x: 1 if x > CUT else 0)

In [26]:
df.describe().T.sort_index()

,count,mean,std,min,25%,50%,75%,max
agriculture,123934.0,0.194112,0.244546,0.000041,0.037912,0.103367,0.233060,0.999335
economics and trade,123934.0,0.320768,0.284428,0.000042,0.079717,0.230730,0.503391,0.998896
education,123934.0,0.151163,0.181124,0.000037,0.040083,0.095900,0.187657,0.999166
environment and climate,123934.0,0.318351,0.279954,0.000050,0.094470,0.230924,0.463007,0.998911
foreign and security affairs,123934.0,0.288059,0.271137,0.000075,0.069461,0.190847,0.438534,0.999188
health,123934.0,0.384880,0.298325,0.000092,0.135382,0.309944,0.589929,0.998705
human rights,123934.0,0.475546,0.318099,0.000138,0.186734,0.417587,0.780818,0.999586
identifierYear,123934.0,2018.449780,3.058206,2008.000000,2016.000000,2018.000000,2021.000000,2025.000000
infered_topic_agriculture,123934.0,0.105588,0.307311,0.000000,0.000000,0.000000,0.000000,1.000000
infered_topic_economics and trade,123934.0,0.252376,0.434378,0.000000,0.000000,0.000000,1.000000,1.000000


# Agregar

In [29]:
df_final = df[['creator', 'document_date', *[f"infered_topic_{var}" for var in topic_vars]]]

df_final.describe().T.sort_index()

,count,mean,std,min,25%,50%,75%,max
infered_topic_agriculture,123934.0,0.105588,0.307311,0.0,0.0,0.0,0.0,1.0
infered_topic_economics and trade,123934.0,0.252376,0.434378,0.0,0.0,0.0,1.0,1.0
infered_topic_education,123934.0,0.048017,0.213804,0.0,0.0,0.0,0.0,1.0
infered_topic_environment and climate,123934.0,0.225297,0.417780,0.0,0.0,0.0,0.0,1.0
infered_topic_foreign and security affairs,123934.0,0.211153,0.408128,0.0,0.0,0.0,0.0,1.0
infered_topic_health,123934.0,0.310020,0.462503,0.0,0.0,0.0,1.0,1.0
infered_topic_human rights,123934.0,0.436079,0.495899,0.0,0.0,0.0,1.0,1.0
infered_topic_infrastructure and industry,123934.0,0.157923,0.364670,0.0,0.0,0.0,0.0,1.0
infered_topic_technology,123934.0,0.143350,0.350431,0.0,0.0,0.0,0.0,1.0


In [33]:
from datetime import datetime

df_final['Y-m'] = df_final['document_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m'))
df_final['Y-w'] = df_final['document_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%Y-W%W'))


df_ym = df_final.copy()
del df_ym['document_date']
df_ym = df_ym.groupby(['creator', 'Y-m']).sum().reset_index()


df_yw = df_final.copy()
del df_yw['document_date']
df_yw = df_yw.groupby(['creator', 'Y-w']).sum().reset_index()

C:\Users\caca_\AppData\Local\Temp\ipykernel_8720\3522264534.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Y-m'] = df_final['document_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m'))
C:\Users\caca_\AppData\Local\Temp\ipykernel_8720\3522264534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Y-w'] = df_final['document_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%Y-W%W'))


In [35]:
df_ym.describe().T.sort_index()

,count,mean,std,min,25%,50%,75%,max
infered_topic_agriculture,44318.0,0.295275,0.786275,0.0,0.0,0.0,0.0,36.0
infered_topic_economics and trade,44318.0,0.705763,1.166732,0.0,0.0,0.0,1.0,42.0
infered_topic_education,44318.0,0.134280,0.443273,0.0,0.0,0.0,0.0,30.0
infered_topic_environment and climate,44318.0,0.630037,1.071155,0.0,0.0,0.0,1.0,23.0
infered_topic_foreign and security affairs,44318.0,0.590482,1.042399,0.0,0.0,0.0,1.0,30.0
infered_topic_health,44318.0,0.866962,1.240962,0.0,0.0,1.0,1.0,21.0
infered_topic_human rights,44318.0,1.219482,1.557367,0.0,0.0,1.0,2.0,42.0
infered_topic_infrastructure and industry,44318.0,0.441626,0.809165,0.0,0.0,0.0,1.0,17.0
infered_topic_technology,44318.0,0.400875,0.764065,0.0,0.0,0.0,1.0,18.0


In [36]:
df_final.to_csv("./data/gold/panel_data_questions_v020703.csv", index=False)
df_ym.to_csv("./data/gold/panel_data_questions_Ym_v020703.csv", index=False)
df_yw.to_csv("./data/gold/panel_data_questions_Yw_v020703.csv", index=False)